In [8]:
# Idea:
# 1. Normalize all integers in the list (divide by its modulus)
# 2. Run Deutsch to see if const (const == only positives or only negatives; balanced 
# => negative numbers exist) 
# 3. Const => Check wether the first number is negative => True

# The function currently works with a fixed small number of qubits only 
# and the calculations may be done in seveal patches. 

In [9]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi

from qiskit import IBMQ, execute
from qiskit.visualization import plot_histogram
from qiskit.providers.basicaer import BasicAer

In [10]:
def prepare_circuit(list_number):
    print("Preparing circuit for ", list_number)
    # Determine the sizes of the quantum and classical registers.
    work_qbits_num = len(list_number)
    ancilla_qbits_num = 1
    all_qbits_num = work_qbits_num + ancilla_qbits_num 
    classical_bits_num = all_qbits_num
    
    # Create quantum and classical registers
    qr = QuantumRegister(work_qbits_num, 'q')
    anc = QuantumRegister(ancilla_qbits_num, 'ancilla')
    cr = ClassicalRegister(classical_bits_num, 'c')
    circuit = QuantumCircuit(anc, qr, cr)
    
    # Encode list into the circuit.
    for i, _ in enumerate(list_number):
        if list_number[i] == -1:
            circuit.x(i)

    circuit.barrier()
    
    # Put all qubits into a superposition
    circuit.h(i for i in range(all_qbits_num))
    
    circuit.barrier()
    
    # Apply CX gate with all targets being the ancilla qubit 
    for i in range(work_qbits_num):  
        circuit.cx(qr[i], anc[0])
    
    circuit.barrier()
    
    # Get back into the initial state in case the list is constant, 
    # or into almost the same state as the initial one, but with the 
    # ancilla qubit having changed its value
    circuit.h(i for i in range(all_qbits_num))
    
    # Measure all qubits
    circuit.measure(anc[0], cr[0])
    for i in range(work_qbits_num):
        circuit.measure(qr[i], cr[i+1])
    
    return circuit

In [11]:
def is_constant(list_number):
    """
    list_number: integer list
    
    To be constant means that all of its elements have the same value 
    """
    circuit = prepare_circuit(list_number)
    circuit.draw()
    
    backend = BasicAer.get_backend("qasm_simulator")
    shots = 1000
    results = execute(circuit, backend=backend, shots=shots).result()
    probabilities = results.get_counts()

    plot_histogram(probabilities)
    
    # We use the QASM simulator => only one answer should be provided by 
    # the circuit with probability 1. In other cases we would use the answer
    # with maximum probablity.
    max_probability = list(probabilities.keys())[0]

    # Our answer is hidden in the ancilla qubit. If it is 0 that would mean that 
    # the list is constant. Otherwise, it is balanced. 
    return max_probability[0] == '0'

In [12]:
def preprocess_data(list_number):
    """
    list_number: integer list
    
    The preprocessing of the data means that all values will be normalized
    (divided by their modulus)
    """
    print("Preprocessing data")
        
    for i, _ in enumerate(list_number):
        try:
            list_number[i] //= abs(list_number[i])
        except ZeroDivisionError:
            # continue
            list_number[i] = 1

    return list_number

In [13]:
def find_negative_numbers(list_number):
    """
    list_number : integer list!.
    Return the True or False depends of the input
    
    The function tracks if the the list is constant, given that all elements
    are either 1 or -1 (or 0 in the specific case where 0 is included in the 
    given list of numbers).
    
    The list to be constant would mean that either all values are 1 (all are 
    positive), or all values are -1 (all are negative). If the function of the 
    values of the list are constant, and the the first number in the list is 
    negative, that would mean that all values in the list are negative. Otherwise, 
    the function of the values of the elements in the list is balanced, and therefore
    it consists of both functions  
    """
    if list_number[0] < 0:
        print("First value is negative:", list_number[0])
        return True
    
    print(list_number)
    list_number = preprocess_data(list_number)
    print(list_number)
     
    return not is_constant(list_number)

In [14]:
# Test: random list with a negative number, first positive number
print("[1,-3,2,15]: ", find_negative_numbers([1,-3,2,15]), "should be: True\n\n")

# Test: all values are positive
print("[1,4,8,11]:", find_negative_numbers([1,4,8,11]), "should be: False\n\n")

# Test: random list, starts with a negative number 
print("[-15,-14,2,-1]:", find_negative_numbers([-15,-14,2,-1]), "should be: True\n\n")

# Test: all values are negative
print("[-15,-14,-2,-1]:", find_negative_numbers([-15,-14,-2,-1]), "should be: True\n\n")

# Test with a zero in them
# Positive first value, 0 somewhere inside
print("[15,-14, 0,-1]:", find_negative_numbers([15,-14, 0,-1]), "should be: True\n\n")
# Negative first value, 0 somewhere inside
print("[-15,-14, 0,-1]:", find_negative_numbers([-15,-14, 0,-1]), "should be: True\n\n")
# Starting with a 0
print("[0, -15,-14,-1]:", find_negative_numbers([0, -15,-14,-1]), "should be: True\n\n")
# All positive plus a 0
print("[15, 14, 0, 1]:", find_negative_numbers([15, 14, 0, 1]), "should be: False\n\n")

[1, -3, 2, 15]
Preprocessing data
[1, -1, 1, 1]
Preparing circuit for  [1, -1, 1, 1]
[1,-3,2,15]:  False should be: True


[1, 4, 8, 11]
Preprocessing data
[1, 1, 1, 1]
Preparing circuit for  [1, 1, 1, 1]
[1,4,8,11]: False should be: False


First value is negative: -15
[-15,-14,2,-1]: True should be: True


First value is negative: -15
[-15,-14,-2,-1]: True should be: True


[15, -14, 0, -1]
Preprocessing data
[1, -1, 1, -1]
Preparing circuit for  [1, -1, 1, -1]
[15,-14, 0,-1]: False should be: True


First value is negative: -15
[-15,-14, 0,-1]: True should be: True


[0, -15, -14, -1]
Preprocessing data
[1, -1, -1, -1]
Preparing circuit for  [1, -1, -1, -1]
[0, -15,-14,-1]: False should be: True


[15, 14, 0, 1]
Preprocessing data
[1, 1, 1, 1]
Preparing circuit for  [1, 1, 1, 1]
[15, 14, 0, 1]: False should be: False


